<h1 align="center"><font size="5">Data Scrapping from Wikipedia</font></h1>

I am going to use BeautifulSoup package for Web Scrapping.

First install the required packages. My environment already has these packages installed.

#!conda install bs4
#!conda install lxml

Lets first load required libraries:

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Use the Requsts package to call the wiki page.

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

wikiPage = requests.get(url).text

Use the Beautiful Soup Package to fetch wiki page contents and find all the tables in it.

In [5]:
PageContent = BeautifulSoup(wikiPage, 'lxml')
TablesList = PageContent.find_all('table')

Loop through the Tables to find required table for this exercise. Right table is found by matching the Headings of the Postal code table.

In [6]:
for table in TablesList:
    Table_Headers = table.find_all('th')
    Headings = [Table_Header.text.strip() for Table_Header in Table_Headers]
    if Headings[:4] == ['Postal code', 'Borough', 'Neighborhood']:
        break

Now fetch column data from tables into 3 different lists.
I have added following logic as per the required data:
 - I have also added the logic to skip rows which do not have 'Not Assigned' value in Borough columns.
 - As the Wiki page does not have duplicate postal code and were already merged with '/' value, so I have replaced '/' with ',' value in the  Neighborhood Column.

In [7]:
Postal_Code=[]
Borough=[]
Neighborhood=[]
for trs in table.findAll('tr'):
    tds=trs.findAll('td')
    if len(tds)==3:
        if tds[1].text.strip()!='Not assigned':
            Postal_Code.append(tds[0].text.strip())
            Borough.append(tds[1].text.strip())
            if (tds[2].text.strip().find('/') != -1):
                Neighborhood.append(tds[2].text.strip().replace("/", ","))
            else:
                Neighborhood.append(tds[2].text.strip())

Create new Data frame and populate the above column data from 3 different lists into Dataframe.

In [9]:
PostalCode_Dataframe=pd.DataFrame(Postal_Code,columns=['Postal_code'])
PostalCode_Dataframe['Borough']=Borough
PostalCode_Dataframe['Neighborhood']=Neighborhood
PostalCode_Dataframe.head(10)

,Postal_code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Finally print the dataframe shape as part of the assignment.

In [10]:
PostalCode_Dataframe.shape

(103, 3)